<a href="https://colab.research.google.com/github/afirez/alphazz-py/blob/main/py_trading_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install websocket-client

In [3]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib

--2021-06-27 06:36:20--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-06-27 06:36:20--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: https://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2021-06-27 06:36:20--  https://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving netactu

In [4]:
import websocket
import talib
import numpy as np
import json

In [25]:
cc = 'btcusdt'
interval = '1m'

socket = f'wss://stream.binance.com:9443/ws/{cc}@kline_{interval}'

In [59]:
# Trading Strategy Parameters

aroon_time_period = 3

core_to_trade = True
core_quantity = 0

amount = 1000
core_trade_amount = amount * 0.80
trade_amounnt = amount * 0.20
money_end = amount
portfolio = 0
investment = []
opens, closes, highs, lows = [], [], [], []
real_time_portfolio_values = []

In [9]:
# Paper trading simulation functions

def buy(allcated_money, price):
  global money_end, portfolio, investment
  quantity = allcated_money / price
  money_end -= quantity * price
  portfolio += quantity
  if investment == []:
    investment.append(allcated_money)
  else:
    investment.append(allcated_money)
    investment[-1] += investment[-2]

def sell(allcated_money, price):
  global money_end, portfolio, investment
  quantity = allcated_money / price
  money_end += quantity * price
  portfolio -= quantity
  investment.append(-allcated_money)
  investment[-1] += investment[-2]

In [60]:
def on_message(ws, message):
  global opens,closes,highs,lows, core_to_trade, core_quantity, money_end, portfolio, investment
  # print(message)
  json_message = json.loads(message)
  cs = json_message['k']
  candle_closed, open, close, high, low = cs['x'], cs['o'], cs['c'], cs['h'], cs['l']
  # print(f'candle_closed: {candle_closed}')
  # print(f'open: {open}')
  # print(f'close: {close}')
  # print(f'high: {high}')
  # print(f'low: {low}')

  if candle_closed:
    opens.append(float(open))
    closes.append(float(close))
    highs.append(float(high))
    lows.append(float(low))
    # print(f'opens: {opens}')
    print(f'closes: {closes}')
    # print(f'highs: {highs}')
    # print(f'lows: {lows}')
    last_price = closes[-1]

    if core_to_trade:
      buy(core_trade_amount, closes[-1])
      print(f'Core investment: We bought ${core_trade_amount} worth of bitcoin', '\n')
      core_quantity += core_trade_amount / closes[-1]
      core_to_trade = False

    aroon = talib.AROONOSC(np.array(highs), np.array(lows), aroon_time_period)
    # print(f'aroon: {aroon}')
    last_aroon = round(aroon[-1], 2)
    # print(f'last_aroon: {last_aroon}')
    amt = last_aroon * trade_amounnt / 100
    port_value = portfolio * last_price - core_quantity * last_price
    trade_amt = amt - port_value
    print(trade_amt)
    print(port_value)

    RT_portfolio_value = port_value + core_quantity * last_price + money_end
    real_time_portfolio_values.append(float(RT_RT_portfolio_value))

    print(f'The Last Aroon is "{last_aroon}" and recommended exposure is "${amt}"')
    print(f'Real-time portfolio value: {RT_RT_portfolio_value}', '\n')
    # print(real_time_portfolio_values)

    if trade_amt > 0: 
      buy(trade_amt, price=last_price)
      print(f'We bought ${trade_amt} worth of bitcoin', '\n', '\n')
    elif trade_amt < 0:
      sell(-trade_amt, price=last_price)
      print(f'We sold ${-trade_amt} worth of bitcoin', '\n', '\n')

def on_error(ws, error):
  print(error)

def on_close(ws, close_status_code, close_msg):
  portfolio_value = portfolio * closes[-1]
  if portfolio_value > 0:
    sell(portfolio_value, price = closes[-1])
  else:
    buy(-portfolio_value, price = closes[-1])
  money_end += investment[-1]

  print("### closed and all trades settled ###")

# def on_open(ws):
#     def run(*args):
#         for i in range(3):
#             time.sleep(1)
#             ws.send("Hello %d" % i)
#         time.sleep(1)
#         ws.close()
#         print("thread terminating...")
#     thread.start_new_thread(run, ())

ws = websocket.WebSocketApp(socket,
                            # on_open=on_open,
                            on_message=on_message,
                            on_error=on_error,
                            on_close=on_close
                            )

In [ ]:
ws.run_forever()

closes: [33069.08, 33023.31]
nan
-798.8927421022901
closes: [33069.08, 33023.31, 33008.7]
nan
-798.539300155916
closes: [33069.08, 33023.31, 33008.7, 32971.11]
597.6299310413232
-797.6299310413232
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81]
665.2019612520214
-798.5419612520215
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81, 33039.23]
932.6178752841023
-799.2778752841023
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81, 33039.23, 33014.7]
998.6844508525788
-798.6844508525788
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81, 33039.23, 33014.7, 33014.56]
932.0210640029901
-798.6810640029901
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81, 33039.23, 33014.7, 33014.56, 33031.14]
665.7421637614351
-799.0821637614351
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81, 33039.23, 33014.7, 33014.56, 33031.14, 33046.73]
866.1193136549309
-799.459313654931
closes: [33069.08, 33023.31, 33008.7, 32971.11, 33008.81, 33039.23, 33014.7, 33014.56, 3

In [41]:
investment

[800.0]

In [42]:
portfolio

0.02408617793603735

In [43]:
money_end

200.0

In [14]:
# Return and Comparison with Benchmark

beg = closes[0]
end = closes[-1]

btc_return = (end/beg-1)
btc_return * 100

In [15]:
bot_return = (money_end/amount - 1)
bot_return * 100

[300, 800]

In [16]:
alpha = bot_return - btc_return
alpha * 100 

10.142857142857142

In [17]:
btc_risk = np.std(np.log(np.array(closes[1:]) / np.array(closes[:-1])))
btc_risk

7.142857142857143

In [18]:
bot_risk = np.std(np.log(np.array(real_time_portfolio_values[1:]) / np.array(real_time_portfolio_values[:-1])))

200.0

In [19]:
btc_sharpe_ratio = btc_return / btc_risk
round(btc_sharpe_ratio, 3)

In [20]:
bot_sharpe_ratio = bot_return / bot_risk
round(bot_sharpe_ratio, 3)

[300, 800, 400]